In [1]:
import numpy as np 
import pandas as pd 
import warnings
import re
warnings.filterwarnings("ignore")
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("dark")
plt.style.use('dark_background')
print("Setup Complete")

Setup Complete


In [2]:
dataset = pd.read_csv("/kaggle/input/microsoft-adventure-works-cycles-customer-data/AWCustomers.csv")
dataset.head()

,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,LastUpdated
0,21173,NaN,Chad,C,Yuan,NaN,7090 C. Mount Hood,NaN,Wollongong,New South Wales,...,Bachelors,Clerical,M,M,1,3,0,1,81916,2017-03-06
1,13249,NaN,Ryan,NaN,Perry,NaN,3651 Willow Lake Rd,NaN,Shawnee,British Columbia,...,Partial College,Clerical,M,M,1,2,1,2,81076,2017-03-06
2,29350,NaN,Julia,NaN,Thompson,NaN,1774 Tice Valley Blvd.,NaN,West Covina,California,...,Bachelors,Clerical,F,S,0,3,0,0,86387,2017-03-06
3,13503,NaN,Theodore,NaN,Gomez,NaN,2103 Baldwin Dr,NaN,Liverpool,England,...,Partial College,Skilled Manual,M,M,1,2,1,2,61481,2017-03-06
4,22803,NaN,Marshall,J,Shan,NaN,Am Gallberg 234,NaN,Werne,Nordrhein-Westfalen,...,Partial College,Skilled Manual,M,S,1,1,0,0,51804,2017-03-06


In [3]:
dataset["AddressLine2"] = dataset["AddressLine2"].fillna("")
dataset["MiddleName"] = dataset["MiddleName"].fillna("")

dataset["Address"] = dataset["AddressLine1"]+" "+dataset["AddressLine2"]+" "+dataset["City"]
dataset["Name"] = dataset["FirstName"]+" "+dataset["MiddleName"]+" "+dataset["LastName"]
dataset["BirthDate"] = pd.to_datetime(dataset["BirthDate"])

In [4]:
colsToDrop = ["AddressLine1","AddressLine2","City","Suffix","Title","MiddleName","FirstName","LastName","PostalCode","LastUpdated"]
dataset.drop(colsToDrop,inplace=True,axis=1)

In [5]:
def cleanPhoneNumber(number):
    number = re.sub('[^0-9]+','', number)
    return int(number)
dataset["PhoneNumber"] = dataset["PhoneNumber"].apply(cleanPhoneNumber) 

In [6]:
dataset.head()

,CustomerID,StateProvinceName,CountryRegionName,PhoneNumber,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,Address,Name
0,21173,New South Wales,Australia,1115005550115,1987-11-13,Bachelors,Clerical,M,M,1,3,0,1,81916,7090 C. Mount Hood Wollongong,Chad C Yuan
1,13249,British Columbia,Canada,7875550136,1972-07-21,Partial College,Clerical,M,M,1,2,1,2,81076,3651 Willow Lake Rd Shawnee,Ryan Perry
2,29350,California,United States,1205550111,1985-11-09,Bachelors,Clerical,F,S,0,3,0,0,86387,1774 Tice Valley Blvd. West Covina,Julia Thompson
3,13503,England,United Kingdom,1115005550167,1977-10-18,Partial College,Skilled Manual,M,M,1,2,1,2,61481,2103 Baldwin Dr Liverpool,Theodore Gomez
4,22803,Nordrhein-Westfalen,Germany,1115005550113,1975-02-05,Partial College,Skilled Manual,M,S,1,1,0,0,51804,Am Gallberg 234 Werne,Marshall J Shan


In [7]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, normalize

lEncoder = LabelEncoder()
ohEncoder = OneHotEncoder()

dataset["CountryRegionName"] = lEncoder.fit_transform(dataset["CountryRegionName"])
dataset["StateProvinceName"] = lEncoder.fit_transform(dataset["StateProvinceName"])
dataset["Education"] = lEncoder.fit_transform(dataset["Education"])
dataset["Occupation"] = lEncoder.fit_transform(dataset["Occupation"])
dfGender = ohEncoder.fit_transform(np.array(dataset["Gender"]).reshape(-1, 1)).toarray()
dfMarital = ohEncoder.fit_transform(np.array(dataset["MaritalStatus"]).reshape(-1, 1)).toarray()
dataset.drop(["Gender","MaritalStatus"],inplace=True,axis=1)

In [8]:
dfMarital = pd.DataFrame(dfMarital)
dfMarital.columns = ["Married","Single"]

dfGender = pd.DataFrame(dfGender)
dfGender.columns = ["Female","Male"]

dataset= pd.concat([dataset,dfGender,dfMarital],axis=1)

### 1. Discrete Columns:  
StateProvinceName, CountryRegionName, PhoneNumber, BirthDate, Education, Occupation, Gender, MaritalStatus, HomeOwnerFlag, NumberCarsOwned, NumberChildrenAtHome, TotalChildren

### 2. Continuous Columns:
YearlyIncome

### 3. Nominal Columns:

### 4. Ordinal Columns:


# Normalization

In [9]:
dataset["YearlyIncome"] = normalize(dataset["YearlyIncome"].to_numpy().reshape(1,-1)).reshape(-1,1)

In [10]:
dataset.head()

,CustomerID,StateProvinceName,CountryRegionName,PhoneNumber,BirthDate,Education,Occupation,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,Address,Name,Female,Male,Married,Single
0,21173,27,0,1115005550115,1987-11-13,0,0,1,3,0,1,0.007656,7090 C. Mount Hood Wollongong,Chad C Yuan,0.0,1.0,1.0,0.0
1,13249,5,1,7875550136,1972-07-21,3,0,1,2,1,2,0.007578,3651 Willow Lake Rd Shawnee,Ryan Perry,0.0,1.0,1.0,0.0
2,29350,6,5,1205550111,1985-11-09,0,0,0,3,0,0,0.008074,1774 Tice Valley Blvd. West Covina,Julia Thompson,1.0,0.0,0.0,1.0
3,13503,8,4,1115005550167,1977-10-18,3,4,1,2,1,2,0.005746,2103 Baldwin Dr Liverpool,Theodore Gomez,0.0,1.0,1.0,0.0
4,22803,30,3,1115005550113,1975-02-05,3,4,1,1,0,0,0.004842,Am Gallberg 234 Werne,Marshall J Shan,0.0,1.0,0.0,1.0
